https://huggingface.co/DeepPavlov/roberta-large-winogrande

chemin jsonl de base : "/home/aurelien/Wino-UROP/train_xl.jsonl"

chemin avec femme et homme inversés : "/home/aurelien/Wino-UROP/sakaguchi/modif_df_inv_genre.jsonl"

chemin avec chaine de caractère chelou : "modif_df_random_mask_noums.jsonl"

In [1]:
import pandas as pd
# Load your dataset
df = pd.read_json("/home/aurelien/Wino-UROP/sakaguchi/modif_df_inv_genre.jsonl", lines=True)
df = df.head(5000)
pd.set_option('display.max_colwidth', None)
df

,qID,sentence,option1,option2,answer
0,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-2,Jill volunteered to eat Jessy's menudo after already having a bowl because _ despised eating intestine.,Jill,Jessy,2
1,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-1,Jill volunteered to eat Jessy's menudo after already having a bowl because _ enjoyed eating intestine.,Jill,Jessy,1
2,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-1,"He never comes to my home, but I always go to her house because the _ is smaller.",home,house,1
3,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-2,"He never comes to my home, but I always go to his house because the _ is bigger.",home,house,2
4,3D5G8J4N5CI2K40F4RZLF9OG2CKVTH-2,"Doris doesn't wear leg warmers to bed, while Sarah almost always does. _ is more likely to live in a colder climate.",Doris,Sarah,2
...,...,...,...,...,...
4995,3GS542CVJVLN86DFGW0RSD5X2LG59H-2,Colby got along well with their boss but Ricky did not because _ had a very rude boss.,Colby,Ricky,2
4996,3HUR21WDDW3Q5F44ECN2BBDHYY1YXO-1,Some tea spilled on the table and Ariana cleaned it with a napkin. The _ got dry.,table,napkin,1
4997,3HUR21WDDW3Q5F44ECN2BBDHYY1YXO-2,Some tea spilled on the table and Ariana cleaned it with a napkin. The _ got wet.,table,napkin,2
4998,3UQ1LLR26CMU93PCKN6YXV0NP9XAL0-2,I just bought a new bag for my camera but it can't fit because the _ is too small.,camera,bag,2


In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

In [3]:
# Config
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Spécifiez le chemin vers le répertoire où le fichier zip a été extrait
model_path = "/home/aurelien/roberta/model_roberta-ft/model_roberta-ft/"

# Charger le tokenizer et le modèle depuis le répertoire local
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Déplacer le modèle vers le bon appareil (CPU ou GPU)
model.to(device)

# Mettre le modèle en mode évaluation
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [5]:
# Préparer les phrases (chaque ligne donne deux phrases à scorer)
option1_sentences = []
option2_sentences = []

for _, row in df.iterrows():
    option1_sentences.append(row["sentence"].replace("_", row["option1"]))
    option2_sentences.append(row["sentence"].replace("_", row["option2"]))

# Liste finale des prédictions
predictions = []

# Boucle par batchs
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Batched predictions"):
    batch1 = option1_sentences[i:i + BATCH_SIZE]
    batch2 = option2_sentences[i:i + BATCH_SIZE]

    # Tokenization groupée
    inputs1 = tokenizer(batch1, padding=True, truncation=True, return_tensors="pt").to(device)
    inputs2 = tokenizer(batch2, padding=True, truncation=True, return_tensors="pt").to(device)

    # Prédictions
    with torch.no_grad():
        logits1 = model(**inputs1).logits
        logits2 = model(**inputs2).logits

        # Conversion en probabilités (modèle binaire à 1 sortie)
        probs1 = torch.sigmoid(logits1.view(-1))  # Forme [BATCH_SIZE]
        probs2 = torch.sigmoid(logits2.view(-1))

        # Comparaison
        batch_preds = (probs2 > probs1).long() + 1  # +1 pour obtenir 1 ou 2
        predictions.extend(batch_preds.cpu().tolist())

# Évaluation
df["predicted"] = predictions
accuracy = (df["predicted"] == df["answer"]).mean()
print(f"Accuracy: {accuracy:.2%}")

Batched predictions: 100%|██████████| 313/313 [01:57<00:00,  2.65it/s]

Accuracy: 99.56%
